In [1]:
import os
import math
import torch
from dataclasses import dataclass, field

from code.train import train
from code.optimizers import Optimizer
# from code.problems import Problem
from code.problem import Loss
from code.datasets import Dataset
from code.models import Model

# %matplotlib widget
%load_ext autoreload
%autoreload 1

In [2]:
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

In [3]:
os.environ['MLFLOW_VERBOSE'] = 'True'
os.environ['MLFLOW_EXPERIMENT_NAME'] = os.path.basename(os.getcwd())

In [4]:
os.path.basename(os.getcwd())

'MeritFed-N'

In [ ]:
m = torch.nn.Dropout1d(p=0.2)
input = torch.randn(2, 2, 5, 5)
input.shape


(*input.shape[:2], -1)

output = m(input.reshape((*input.shape[:2], -1)))
output

In [36]:
@dataclass
class Config():
    n_iters: int = 2000
    n_peers: int = 20
    seed: int = 0

    model: Model = field(default_factory=lambda: Model.Net)
    loss: Loss = field(default_factory=lambda: Loss.CrossEntropy)

    dataset: Dataset = field(default_factory=lambda: Dataset.MNIST)

#     model: Model = field(default_factory=lambda: Model.Mean)
#     loss: Loss = field(default_factory=lambda: Loss.MSE)

#     dataset: Dataset = field(default_factory=lambda: Dataset.Normal)
    n_samples: int = 200
    h_ratio: float = 0.99
    mu_normal: float = None

    optimizer: Optimizer = field(default_factory=lambda: Optimizer.SGD)
    batch_size: int = 20
    lr: float = 1e-2

    true_weights: bool = None

    md_n_iters_: int = None
    md_full_: bool = None
    md_lr_: int = None

In [ ]:
config = Config()
config.optimizer = Optimizer.SGD
config.true_weights = True
os.environ['MLFLOW_RUN_NAME'] = config.optimizer.name
%time train(config)

Trying port 5317
test len  200
3957
full test len  3957
full test len  20
calc1 200
actual3 18
calc3 18
actual1 200


In [ ]:
config = Config()
config.optimizer = Optimizer.MeritFed
config.md_full_ = True
config.md_n_iters_ = 20
config.md_lr_ = 0.01
os.environ['MLFLOW_RUN_NAME'] = config.optimizer.name
%time train(config)

In [ ]:
config = Config()
config.optimizer = Optimizer.SGD
config.true_weights = False
os.environ['MLFLOW_RUN_NAME'] = config.optimizer.name
%time train(config)

In [ ]:
config = Config()
config.optimizer = Optimizer.MeritFed
config.md_full_ = False
config.md_n_iters_ = 20
config.md_lr_ = 0.01
os.environ['MLFLOW_RUN_NAME'] = config.optimizer.name
%time train(config)